In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
!pip install transformers accelerate triton.ops

ERROR: Could not find a version that satisfies the requirement triton.ops (from versions: none)
ERROR: No matching distribution found for triton.ops


In [3]:
! pip install accelerate transformers peft

In [4]:
# !pip uninstall -y bitsandbytes triton
!pip install bitsandbytes triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.1 MB/s eta 0:00:00


In [5]:
import bitsandbytes as bnb
import triton
import torch
print("bitsandbytes:", bnb.__version__)
print("triton:", triton.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

bitsandbytes: 0.48.2
triton: 3.4.0
CUDA available: True
GPU: Tesla T4


In [5]:
from huggingface_hub import login
login("your_hf_permission_token")  # 👉 paste your Hugging Face access token when prompted


In [ ]:
# ------------------------------------------------------------
# 🦙 Llama-3-8B-Instruct – QLoRA Chatbot (Clean, Full Display)
# ------------------------------------------------------------
import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
import re

# ------------------------------------------------------------
# 1. Model Setup
# ------------------------------------------------------------
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

print("🔹 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ✅ QLoRA (4-bit quantization)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print("🔹 Loading quantized model (QLoRA 4-bit)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("\n🤖 Chatbot Loaded Successfully!\n")

# ------------------------------------------------------------
# 2. Token Filters
# ------------------------------------------------------------
SYSTEM_PROMPT = "You are a helpful and knowledgeable assistant."
history_token_total = 0

UNNECESSARY = {
    '', '\n', '<', '|', '>', 'system', 'user', 'assistant',
    '_', 'start', 'end',
    '<|begin_of_text|>', '<|eot_id|>', '<|end_of_text|>',
    '<|end_header_id|>', '<|start_header_id|>', ' '
}
SPECIAL_PATTERN = re.compile(r"(<\|.*?\|>|Ċ|Ġ| |:|>|<|\\n)")

def clean_tokens(tokens):
    """Remove special or unwanted tokens."""
    cleaned = []
    for t in tokens:
        t = SPECIAL_PATTERN.sub("", t).strip()
        if t and t not in UNNECESSARY:
            cleaned.append(t)
    return cleaned


# ------------------------------------------------------------
# 3. Chat Function
# ------------------------------------------------------------
def chat_fn(message, history):
    global history_token_total

    if message.strip().lower() in {"exit", "quit"}:
        history.append((message, "👋 Goodbye! Chat session ended."))
        return history, 0, 0, history_token_total, 0, "", ""

    # 🧠 Build conversation context
    conversation = tokenizer.apply_chat_template(
        [{"role": "system", "content": SYSTEM_PROMPT}]
        + [{"role": "user", "content": u, "role": "assistant", "content": a} for u, a in history]
        + [{"role": "user", "content": message}],
        tokenize=False
    )

    # 1️⃣ Encode + count input tokens
    input_ids = tokenizer(conversation, return_tensors="pt").input_ids
    input_tokens = input_ids.shape[1]
    input_token_text = clean_tokens(tokenizer.convert_ids_to_tokens(input_ids[0]))

    # 2️⃣ Generate response (ensure full, readable output)
    outputs = pipe(
        conversation,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_text = outputs[0]["generated_text"]

    # Extract clean assistant response
    if "Assistant:" in generated_text:
        reply = generated_text.split("Assistant:")[-1].strip()
    elif "assistant" in generated_text.lower():
        reply = generated_text.split("assistant")[-1].strip(": ").strip()
    else:
        reply = generated_text.strip()

    # Make sure the reply ends cleanly (avoid incomplete cutoffs)
    reply = re.sub(r"(\n)+$", "", reply).strip()

    # 3️⃣ Count output tokens
    output_ids = tokenizer(reply, return_tensors="pt").input_ids
    output_tokens = output_ids.shape[1]
    output_token_text = clean_tokens(tokenizer.convert_ids_to_tokens(output_ids[0]))

    # 4️⃣ Update totals
    history_token_total += input_tokens + output_tokens
    total_tokens = input_tokens + output_tokens

    # 🗨️ Show message + reply in UI
    history.append((message, reply))

    return (
        history,
        input_tokens,
        output_tokens,
        history_token_total,
        total_tokens,
        str(input_token_text),
        str(output_token_text),
    )


# ------------------------------------------------------------
# 4. Gradio UI
# ------------------------------------------------------------
with gr.Blocks(title="🦙 Llama-3 QLoRA Chatbot (Full)") as demo:
    gr.Markdown(
        """
        # 🦙 Llama-3-8B QLoRA Chatbot
        **Now with proper responses and visible messages!**
        Tracks: Input Tokens • Output Tokens • History Tokens • Total Tokens
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                label="Chat History",
                height=500,
                bubble_full_width=False,
                show_label=True
            )
            txt = gr.Textbox(
                placeholder="Type your question and press Enter...",
                label="💬 Message"
            )

        with gr.Column(scale=1):
            in_tok = gr.Number(label="🧩 Input Tokens", value=0, interactive=False)
            out_tok = gr.Number(label="🧠 Output Tokens", value=0, interactive=False)
            hist_tok = gr.Number(label="📜 History Tokens", value=0, interactive=False)
            total_tok = gr.Number(label="🔢 Total Tokens", value=0, interactive=False)

            gr.Markdown("### 🔍 Clean Token Display")
            in_tok_text = gr.Textbox(label="📥 Input Tokens (cleaned)", lines=4, interactive=False)
            out_tok_text = gr.Textbox(label="📤 Output Tokens (cleaned)", lines=4, interactive=False)

    def clear_fn():
        global history_token_total
        history_token_total = 0
        return [], "", 0, 0, 0, 0, "", ""

    clear_btn = gr.Button("🧹 Clear Chat")

    txt.submit(
        chat_fn,
        [txt, chatbot],
        [chatbot, in_tok, out_tok, hist_tok, total_tok, in_tok_text, out_tok_text],
    )

    clear_btn.click(
        clear_fn,
        None,
        [chatbot, txt, in_tok, out_tok, hist_tok, total_tok, in_tok_text, out_tok_text],
    )

# ------------------------------------------------------------
# 5. Launch App
# ------------------------------------------------------------
if __name__ == "__main__":
    demo.launch(debug=True, share=False)
